In [27]:
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.tag import pos_tag
import re
import spacy
from nltk import word_tokenize
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
from pattern.en import conjugate, lemma, lexeme, INFINITIVE, PRESENT, PAST, PARTICIPLE, FUTURE, SG, PL, INDICATIVE, IMPERATIVE, CONDITIONAL, SUBJUNCTIVE, PROGRESSIVE, singularize

In [29]:
def VB_VB_correction(payload, raw_text, error_count): # correct errors of type is-walking OR has-cooked
    verbs=['VB','VBD','VBG','VBN','VBP','VBZ']
    past = ['yesterday', 'last_week', 'last_month', 'last_year''ago', 'once', 'once_upon_a_time', 'in_those_days',
            'one_day', 'then', 'in_olden_days', 'in_ancient_times', 'in_early_days', 'in_my_childhood',
            'in_my_boyhood_days', 'today_morning']
    simple_present = ['every_day', 'every_month', 'every_week', 'every_year', 'always', 'sometimes', 'never',
                      'frequently']
    #if(payload.tag_[:2] != 'VB'):
        #print("here")
        #return raw_text, error_count
    nounBeforeVerb = False
    verbFound = False
    grtokens=False
    #i=0
    #print(len(payload))
    tokens = pos_tag(word_tokenize(raw_text))
    for i in range(0, len(tokens)):
        # print(tokens[i][0])
        if (tokens[i][0] in past):
            # print("in loop")
            grtokens = True
    # check if its present simple
    for i in range(0, len(tokens)):
        # print(tokens[i][0])
        if (tokens[i][0] in simple_present):
            grtokens = True
    #length=len(payload)-1
    if(payload.text == 'is' or payload.text == 'was' or payload.text == 'are' or payload.text == 'were' or payload.text == 'were'):
        
        return raw_text, error_count 
    for ch in payload.children:
        if(ch.tag_[:2] in verbs):
           
            verbFound = True
            
        if(grtokens==True and payload is not None):
            
            x=grammartokens(tokens)
            if(x is None):
                return raw_text,0
            listToStr = ' '.join([str(elem) for elem in x]) 
            return listToStr, 1
            
        if((verbFound == False) and((ch.dep_ == 'nsubjpass')or(ch.dep_ == 'nsubj'))):
            nounBeforeVerb = True
        #print(ch.tag_[:2])
        
        if(ch.tag_[:2] in verbs or ch.tag_[:2]=='MD' or payload is not None): # this might need to be removed
            
            ifHave = False
            ifBeen = False
            if((ch.lower_ == 'has') or (ch.lower_ == 'have') or (ch.lower_ == 'had')):
                ifHave = True
            if(ch.lower_ == 'been'):
                ifBeen = True
            try:
                if(ifHave):
                    
                    x = conjugate(verb=lemma(payload.text), tense=PAST+PARTICIPLE, mood=INDICATIVE, person=1, number=PL)
                elif(ifHave and ifBeen):
                   
                    x = conjugate(verb=lemma(payload.text), tense=PAST+PARTICIPLE, mood=INDICATIVE, person=1, number=PL)
                elif(nounBeforeVerb==True  and ((ch.lower_ == 'is') or (ch.lower_ == 'am') or (ch.lower_ == 'are') or (ch.lower_ == 'was')  or (ch.lower_ == 'were'))):
                    x = conjugate(verb=lemma(payload.text), tense=PRESENT, mood=INDICATIVE, aspect=PROGRESSIVE, person=1, number=PL)
                elif(nounBeforeVerb==True  and ((ch.lower_ == 'yesterday') or (ch.lower_ == 'ago'))):
                    
                   
                    x = lexeme(payload.text)[3]
                elif(nounBeforeVerb==True  and ((ch.lower_ == 'next') or (ch.lower_ == 'future') or (ch.lower_ == 'will')or (ch.tag_[:2] == 'MD') )):
                    
                    x = conjugate(verb=lemma(payload.text), tense= INFINITIVE, mood=INDICATIVE, aspect=PROGRESSIVE, person=1, number=PL)    
                else:
                    x = payload.text
                if(x != payload.text):
                    error_count += 1
                raw_text = raw_text[:payload.idx] + raw_text[payload.idx:].replace(payload.text, x, 1)
            except TypeError:
                print("error")
    return raw_text, error_count

In [30]:
def VB_VB_VB_correction(payload,raw_text, error_count): # correct errors of type has-been-walking
    if 'been' not in raw_text.split():
        return raw_text, error_count
    if(payload.tag_[:2] != 'VB' and payload.tag_[:2] != 'NN'  and payload.tag_[:2] != 'JJ'):
        return raw_text, error_count
    for ch in payload.children:
        if(ch.tag_[:2] == 'VB'): # this might need to be removed
            dummy, error_count = VB_VB_VB_correction(ch, raw_text, error_count)
    temp = []
    combos=[]
    nounBeforeVerb = False
    nounAfterVerb = False
    verbFound = False
    since = False
    for ch in payload.children:
        if(ch.tag_[:2] == 'VB'):
            verbFound = True
        if((not verbFound) and (ch.tag_[:2] == 'NN' or ch.tag_[:2] == 'PR')):
            nounBeforeVerb = True
        if(verbFound and (ch.tag_[:2] == 'NN' or ch.tag_[:2] == 'PR')):
            nounAfterVerb = True
        if(ch.lower_ == 'since'):
            since = True
    for ch in payload.children:
        if(ch.tag_[:2] == 'VB'):
            temp.append(ch.lower_ + '_' + ch.tag_)
        if(len(temp) == 2):
            temp.append(payload.lower_+ '_' + ch.tag_)
            try:
                if (temp[0][-3:] == 'VBZ' or temp[0][-3:] == 'VBP') and temp[1][-3:] == 'VBN':
                    if nounAfterVerb or since:
                        x = conjugate(verb=lemma(temp[2][:-4]), tense=PRESENT, mood=INDICATIVE, aspect=PROGRESSIVE, person=1, number=PL)
                    elif nounBeforeVerb:
                        x = conjugate(verb=lemma(temp[2][:-4]), tense=PAST+PARTICIPLE, mood=INDICATIVE, person=1, number=PL)
                else:
                    x = conjugate(verb=lemma(temp[2][:-4]), tense=PRESENT, mood=INDICATIVE, aspect=PROGRESSIVE, person=1, number=PL)
                combos.append(temp)

                if(x != temp[2][:-4]):
                    error_count += 1
                raw_text = raw_text[:payload.idx] + raw_text[payload.idx:].replace(temp[2][:-4], x, 1)
                temp = []
                return raw_text, error_count
            except TypeError:
                return raw_text, error_count
    return raw_text, error_count

In [31]:
def grammartokens(tokens):
    time = 'null'
    verbs=['VB','VBD','VBG','VBN','VBP','VBZ']
    nouns=['NN','NNS','NNP','NNPS','PRP']
    simple_present = ['every_day', 'every_month', 'every_week', 'every_year', 'always', 'sometimes', 'never',
                      'frequently']
    correct = False
    mylist1=None 
    mylist=None
    past = ['yesterday', 'last_week', 'last_month', 'last_year''ago', 'once', 'once_upon_a_time', 'in_those_days',
            'one_day', 'then', 'in_olden_days', 'in_ancient_times', 'in_early_days', 'in_my_childhood',
            'in_my_boyhood_days', 'today_morning']
    modal_words_past=['could not','could','might','might not','did not','did not','should','should not','would','would not','must','must not']
    modal_words_present=['can not','can','may','may not','does not','do not','shall','shallnot','will','will not','must','must not']
    # check if its past
    for i in range(0, len(tokens)):
        # print(tokens[i][0])
        if (tokens[i][0] in past):
            # print("in loop")
            time = 'past'
    # check if its present simple
    for i in range(0, len(tokens)):
        # print(tokens[i][0])
        if (tokens[i][0] in simple_present):
            time = 'simple_present'
    #print(time)
    # check past sentence is correct or no
    #elif((tokens[0][1]=='WRB'or tokens[0][1]=='WP'))
    if (time == 'past'):
                
        for x in range(0, len(tokens)):
            if(tokens[x][1]=='MD' or tokens[x][0]=='did' or tokens[x][0]=='does' or tokens[x][0]=='do'):
                getmidindex=x
                break
        #print(getmidindex)
        #getverbindex
        for i in range(0, len(tokens)):
            if((tokens[i][1] == "VB")or (tokens[i][1] == "VBD") or (tokens[i][1] == "VBG")
               or (tokens[i][1] == "VBN")or (tokens[i][1] == "VBZ")or tokens[i][1] == "VBP"):
                if(tokens[i][0]=='did'):
                    continue
                getverbindex=i
                break
        #print(getverbindex)
        #check sentence or question                                         
        for i in range(0, len(tokens)):
            k=i-1
            c=k-1
            length=len(tokens)-1
            if((tokens[0][1]=='WRB'or tokens[0][1]=='WP')and (tokens[getmidindex][0] in modal_words_past or tokens[i][0]=='did')and
              tokens[getverbindex][1]=='VB'and tokens[length][0]=='?'):
                correct=True
                
            elif((tokens[0][1]=='WRB'or tokens[0][1]=='WP')):
                
                mylist1 = dict(tokens)
                mylist1 = list(mylist1.keys())
                if(tokens[getmidindex][0] not in modal_words_past):
                    
                    if(tokens[getmidindex][0] in modal_words_present):
                        
                        mylist1[getmidindex]=modal_words_past[modal_words_present.index(tokens[getmidindex][0])]
                        
                    else:
                        mylist1[getmidindex]='did'
                        
                if(tokens[getverbindex][1]!='VB'):
                    mylist1[getverbindex]=lexeme(tokens[getverbindex][0])[0]
                    
                if(tokens[length][0]!='?'):
                    if(tokens[length][0]!='.'):
                        mylist1[length]='?'
                    else:
                        mylist1.append("?")
                correct=True
            
            #sentence is correct condition
            elif (tokens[i][1] == "VB" and time == 'past' and (tokens[k][0] =='did'or tokens[c][0] =='did'or tokens[k][1] =='MD'or tokens[c][1] =='MD' )):
                correct = True
                break
            #verb not in base form after modal or didnot
            elif (tokens[i][1] != "VB" and time == 'past' and (tokens[k][0] =='did'or tokens[c][0] =='did'or tokens[k][1] =='MD'or tokens[c][1] =='MD' )):
                correct = False
                break
                
            elif(tokens[i][1] == "VB" and time == 'past' 
                 and(tokens[k][1]=='MD' or tokens[c][1]=='MD' )):
                mylist1 = dict(tokens)
                mylist1 = list(mylist1.keys())
                if(tokens[k][1]=='MD'):
                    if(tokens[k][0]  in modal_words_present):
                        mylist1[k]=modal_words_past[modal_words_present.index(tokens[k][0])]
                if(tokens[c][0]=='MD'):
                    if(tokens[c][0]  in modal_words_present):
                        mylist1[c]=modal_words_past[modal_words_present.index(tokens[c][0])]    
                mylist1[i]=lexeme(tokens[i][0])[0]
                correct=True
                #print(mylist1)
            elif(tokens[i][1] == "VBD" and time == 'past'):
                correct=True
                
            
                # check present simple is correct or no present
    if (time == 'simple_present'):
        
        for x in range(0, len(tokens)):
            k=i-1
            c=k-1
            length=len(tokens)-1
            if(tokens[x][1]=='MD' or tokens[x][0]=='did' or tokens[x][0]=='does' or tokens[x][0]=='do'):
                getmidindex=x
                break
        #print(getmidindex)
        #getverbindex
        
        for i in range(0, len(tokens)):
            if((tokens[i][1] == "VB")or (tokens[i][1] == "VBD") or (tokens[i][1] == "VBG")
               or (tokens[i][1] == "VBN")or (tokens[i][1] == "VBZ") or tokens[i][1] == "VBP" ):
                if(tokens[i][0]=='do' or tokens[i][0]=='does'):
                    continue
                
                getverbindex=i
                break
        #print(getverbindex)
        for i in range(0, len(tokens)):
            if((tokens[0][1]=='WRB'or tokens[0][1]=='WP')and (tokens[getmidindex][0] in modal_words_past or tokens[i][0]=='do' or tokens[i][0]=='does')and
              tokens[getverbindex][1]=='VB'and tokens[length][0]=='?'):
                correct=True
                
            elif((tokens[0][1]=='WRB'or tokens[0][1]=='WP')):
                
                mylist1 = dict(tokens)
                mylist1 = list(mylist1.keys())
                if(tokens[getmidindex][0] not in modal_words_present):
                    
                    if(tokens[getmidindex][0] in modal_words_past):
                        
                        mylist1[getmidindex]=modal_words_present[modal_words_past.index(tokens[getmidindex][0])]
                        
                    elif((tokens[getmidindex+2][1]=="NNP" or tokens[getmidindex+2][1] == "NN" or tokens[getmidindex+2][0]=='he'or tokens[getmidindex+2][0]=='she' or tokens[getmidindex+2][0]=='it')or(tokens[getmidindex+1][1]=="NNP" or tokens[getmidindex+1][1] == "NN" or tokens[getmidindex+1][0]=='he'or tokens[getmidindex+1][0]=='she' or tokens[getmidindex+1][0]=='it')):
                        mylist1[getmidindex]='does'
                    else:
                        
                        mylist1[getmidindex]='do'
                
                if(tokens[getverbindex][1] in verbs):
                    
                    
                    mylist1[getverbindex]=lexeme(tokens[getverbindex][0])[0]
                    
                if(tokens[length][0]!='?'):
                    if(tokens[length][0]=='.'):
                        mylist1[length]='?'
                    else:
                        
                        mylist1.append("?")
                
                correct=True
                
            for j in range(0,len(tokens)):
                if ((tokens[i][1] == "VBP" and (tokens[j][1] == "NNP" or tokens[j][1] == "NN")) or (tokens[i][1] == "VBZ" and (tokens[j][1] == "NNPS" or tokens[j][1] == "NNS"))):              
                     correct = True
    #print(correct)
    #print(mylist1)
    #past sentence correct
    #print(correct)
    if (correct == False and time == 'past'):
                mylist = dict(tokens)
                mylist = list(mylist.keys())
                for i in range(0, len(tokens)):
                    if(tokens[i][0]=='did'or tokens[i][0]in modal_words_past):
                        continue
                    if(tokens[i][1] in verbs):
                        verb = lexeme(tokens[i][0])
                        if(mylist[i]==verb[3] or mylist[i-1]=='not' ):
                            mylist[i]=verb[0]
                            correct=True
                            break
                        else:
                            mylist[i] = verb[3]
                            correct=True
                           
                
                        #print(mylist)
    #present simple correction
    if (correct == False and time == 'simple_present'):
                mylist = dict(tokens)
                mylist = list(mylist.keys())
                #get noun
                for i in range(0, len(tokens)):
                    p=i+1
                    if(tokens[i][1]in nouns and (tokens[p][1] in verbs or tokens[p][1] =='MD' or (tokens[p][1]=='RB' and tokens[++p][1] in verbs ))):
                        savei=i
                        #print(savei)
                        break
                    
                #get verb
                for k in range(0, len(tokens)):
                    if(tokens[k][1] in verbs or tokens[k][1] =='MD' ):
                        verb = lexeme(tokens[k][0])
                        savek=k
                        break
                #correct verb
               # if(tokens[savei+1][0]in modal_words_present[:8] or tokens[savei+1][0]in modal_words_past[:8]):
                    
                    
                if(tokens[savek+1][0]=='not'):
                    mylist[savek+2]=lexeme(tokens[savek+2][0])[0]
                if(tokens[savek+1][1] in verbs):
                    mylist[savek+1]=lexeme(tokens[savek+1][0])[0]
                    
                if(tokens[savei][1]=='NN' or tokens[savei][1]=='NNP'or tokens[savei][0]=='he'or tokens[savei][0]=='she' or tokens[savei][0]=='it'):
                    mylist[savek] = verb[1]
                    if(tokens[savek][1] =='MD'):
                        mylist[savek] = verb[0]
                    
                    
                else:
                    
                    mylist[savek]=verb[0]
    if(mylist is None):
        return mylist1
    else:
        return mylist
          
    #print(mylist1)
               
                
            

In [46]:
en_nlp = spacy.load('en')
def modify(text):
    vv_errors = 0
    vvv_errors = 0
    doc = en_nlp(text)
    punctFound = False
    for token in doc:
        if(token.dep_ == 'punct'):
            punctFound = True
    if not punctFound:
        text +=''
    for sent in doc.sents:
        text, e1 = VB_VB_correction(sent.root, text, 0)
        text, e2 = VB_VB_VB_correction(sent.root, text, 0)
        vv_errors += e1
        vvv_errors += e2
    errors =vv_errors+vvv_errors
    return text, errors

In [55]:
    # measure Performance with some sentences with accuarcy  
    wronglist=['he is go to school',
               'he have been go to school',
               'he will goes to school',
               'he go to school every_day',
               'he goes to school yesterday',
               'Harry has been watched movie since afternoon',
               'she is go to school',
               'she have been reach to school',
               'she should goes to school',
               'she went to school every_day',
               'she gone to school yesterday',
               'Harry has been went to school',
               'she have been reach to school',
               'she should goes to school',
               'she went to school every_day',
               'she gone to school yesterday',
               'Harry has been went to school',
              'Hello world. Hello, this is a sample document. I am sit here in the lab. John is played in the hostel. Harry has been go to the company since last month. There has been tremendous progress in computer science in the last few decades. And credits goes to the rise of develpoment in electronics. The key players in this domain have been competing against each other. Apple is dominate the market with their mobile phones. ']
    correctlist=['he is going to school',
               'he have been gone to school',
               'he will go to school',
               'he goes to school every_day',
               'he went to school yesterday',
               'Harry has been watching movie since afternoon',
               'she is going to school',
               'she have been reached to school',
               'she should go to school',
               'she goes to school every_day',
               'she went to school yesterday',
               'Harry has been gone to school',
               'she have been reached to school',
               'she should go to school',
               'she goes to school every_day',
               'she went to school yesterday',
               'Harry has been gone to school',
                'Hello world. Hello, this is a sample document. I am sit here in the lab. John is playing in the hostel. Harry has been goingne to the company since last month. There has been tremendous progress in computer science in the last few decades. And credits goes to the rise of develpoment in electronics. The key players in this domain have been competed against each other. Apple is dominate the market with their mobile phones. ']
    correcttedlist=[]
    test=[]
    result=[]
    allerrors=0
    for sent in wronglist :
        text,errors = modify(sent)
        allerrors=errors+allerrors 
        correcttedlist.append(text)
    for i in range(len(correcttedlist)):
        if correcttedlist[i]==correctlist[i]:
            result.append(1)
        else:
            result.append(0)
    accuracy = result.count(1)/len(result)*100
    print('accuracy = '+str("%.2f" %accuracy)+'%'+' and number of errors ='+str(allerrors))

accuracy = 83.33% and number of errors =27


In [64]:
#Experimental results read from pdf file and correct sentences from it 
import PyPDF2    
# creating a pdf file object  
pdfFileObj = open('test.pdf', 'rb')  
# creating a pdf reader object  
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   
pageObj = pdfReader.getPage(0)    
text=pageObj.extractText()  
pdfFileObj.close()  

In [65]:
#text from the file
text

'Hello world. Hello, this is a sample document. I am sit here in the lab. John is played in the \nhostel. Harry has been go to the company since last month. There has been tremendous \nprogress in \nco\nmputer science in the last few\n \ndecades. And credits goes to t\nhe rise of \ndevelpoment in electronics. The key players in this domain have been competing against \neach other. Apple is dominate the market with their mobile phones.\n \n'

In [66]:
%%time
#delete \n and correct grammer and return the text and 
text=re.sub("\n","",text)
modify(text)


Wall time: 77.8 ms


('Hello world. Hello, this is a sample document. I am sit here in the lab. John is playing in the hostel. Harry has been goingne to the company since last month. There has been tremendous progress in computer science in the last few decades. And credits goes to the rise of develpoment in electronics. The key players in this domain have been competed against each other. Apple is dominate the market with their mobile phones. ',
 5)

In [67]:
modify('he is play football')

('he is playing football', 1)